In [1]:
from importlib import reload
import watermarking.watermark_processor

watermarking.watermark_processor = reload(watermarking.watermark_processor)
from watermarking.watermark_processor import RepetitionPenaltyLogitsProcessor
from transformers import LogitsProcessorList, MinLengthLogitsProcessor, LogitsProcessor
from watermarking.utils.text_tools import truncate
from watermarking.utils.load_local import load_local_model_or_tokenizer
from datasets import load_dataset, load_from_disk

In [2]:
tokenizer = load_local_model_or_tokenizer("facebook/opt-1.3b", 'tokenizer')
model = load_local_model_or_tokenizer("facebook/opt-1.3b", 'model')

loading opt-1.3b tokenizer from /home/wanglean/llm-ckpts/tokenizers/opt-1.3b ...
finished loading
loading opt-1.3b model from /home/wanglean/llm-ckpts/models/opt-1.3b ...
finished loading


In [466]:
import os
os.environ['CURL_CA_BUNDLE'] = ''

from transformers import AutoTokenizer, AutoModelForCausalLM
lm_tokenizer = AutoTokenizer.from_pretrained('gpt2')
lm_model = AutoModelForCausalLM.from_pretrained('gpt2').to('cuda:0')

SSLError: HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /gpt2/resolve/main/tokenizer_config.json (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:997)')))

In [7]:
model = model.to('cuda:0')

In [8]:
c4_sliced_and_filted = load_from_disk('./c4-train.00000-of-00512_sliced')
c4_sliced_and_filted = c4_sliced_and_filted['train'].shuffle(seed=args.sample_seed).select(
    range(100))

Loading cached shuffled indices for dataset at /home/wanglean/wm/c4-train.00000-of-00512_sliced/train/cache-bf7c139e2a7a4790.arrow


In [396]:
# 选择一个prompt

sample_idx = 98
input_text = c4_sliced_and_filted[sample_idx]['text']
tokenized_input = tokenizer(input_text, return_tensors='pt').to(model.device)
tokenized_input = truncate(tokenized_input, max_length=300)

In [10]:
min_length_processor = MinLengthLogitsProcessor(min_length=1000,
                                                eos_token_id=tokenizer.eos_token_id)
repetition_processor = RepetitionPenaltyLogitsProcessor(penalty=1.5)

In [253]:
from importlib import reload
import watermarking.watermark_processors.message_models.lm_message_model
watermarking.watermark_processors.message_models.lm_message_model = reload(watermarking.watermark_processors.message_models.lm_message_model)
import watermarking.watermark_processors.message_model_processor
watermarking.watermark_processors.message_model_processor = reload(watermarking.watermark_processors.message_model_processor)


from watermarking.watermark_processors.message_models.lm_message_model import LMMessageModel
from watermarking.watermark_processors.message_model_processor import WmProcessorMessageModel

In [449]:
lm_message_model = LMMessageModel(tokenizer=tokenizer,lm_model=model,lm_tokenizer=tokenizer,
    delta = 1.1, lm_prefix_len=10, lm_topk=-1, message_code_len = 20,random_permutation_num=50)
wm_precessor_message_model = WmProcessorMessageModel(message_model=lm_message_model,tokenizer=tokenizer,
    encode_ratio=5,max_confidence_lbd=0.5,strategy='max_confidence', message=[42,34,54,665,226,329])

In [450]:
# 生成带watermark的文本

start_length = tokenized_input['input_ids'].shape[-1]
wm_precessor_message_model.start_length = start_length
output_tokens = model.generate(**tokenized_input, max_new_tokens=200, num_beams=4,
                               logits_processor=LogitsProcessorList(
                                   [min_length_processor, repetition_processor,
                                    wm_precessor_message_model]))
output_text = tokenizer.decode(output_tokens[0][tokenized_input['input_ids'].shape[-1]:],
                               skip_special_tokens=True) # 生成的文本
prefix_and_output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True) # 带prompt的文本

In [ ]:
log_probs = wm_precessor_message_model.decode(output_text)

In [ ]:
log_probs[0] # message

In [431]:
# 定义random watermark
import watermarking.watermark_processors.random_processor
watermarking.watermark_processors.random_processor = reload(watermarking.watermark_processors.random_processor)

from watermarking.watermark_processors.random_processor import WmProcessorRandom
random_processor = WmProcessorRandom(message=[42,34,5465,34], tokenizer=tokenizer,delta=1.5,message_code_len = 20,
     top_k=100, encode_ratio = 5.
    )

In [456]:
# 生成带random watermark的文本
random_processor.start_length = tokenized_input['input_ids'].shape[-1]
t_output_tokens = model.generate(**tokenized_input, max_new_tokens=200, num_beams=4,
                               logits_processor=LogitsProcessorList(
                                   [min_length_processor, repetition_processor,
                                    random_processor]))
t_output_text = tokenizer.decode(t_output_tokens[0][tokenized_input['input_ids'].shape[-1]:],
                               skip_special_tokens=True) # 生成的文本
t_prefix_and_output_text = tokenizer.decode(t_output_tokens[0], skip_special_tokens=True) # 带prompt的文本

In [457]:
t_log_probs = random_processor.decode(t_output_text)

100%|██████████| 200/200 [00:00<00:00, 1537.55it/s]


In [458]:
t_log_probs[0] # message

[tensor(42, device='cuda:0'), tensor(34, device='cuda:0')]

In [430]:
# 生成不带time watermark的文本

no_wm_output_tokens = model.generate(**tokenized_input, max_new_tokens=200, num_beams=4,
                                     logits_processor=LogitsProcessorList(
                                       [min_length_processor, repetition_processor]))
no_wm_output_text = tokenizer.decode(no_wm_output_tokens[0][tokenized_input['input_ids'].shape[-1]:],
                                     skip_special_tokens=True)
no_wm_prefix_and_output_text = tokenizer.decode(no_wm_output_tokens[0], skip_special_tokens=True)

In [33]:
oracle_tokenizer = load_local_model_or_tokenizer('facebook/opt-2.7b', 'tokenizer')
oracle_model = load_local_model_or_tokenizer('facebook/opt-2.7b', 'model')
oracle_model = oracle_model.to('cuda:0')

loading opt-2.7b tokenizer from /home/wanglean/llm-ckpts/tokenizers/opt-2.7b ...
finished loading
loading opt-2.7b model from /home/wanglean/llm-ckpts/models/opt-2.7b ...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

finished loading


In [454]:
from watermarking.experiments.watermark import compute_ppl_single

# 计算带水印的ppl
loss, ppl = compute_ppl_single(prefix_and_output_text=prefix_and_output_text,
                               oracle_model_name=args.oracle_model_name,
                               output_text=output_text,
                               oracle_model=oracle_model, oracle_tokenizer=oracle_tokenizer)
print(loss, ppl)

1.2013373374938965 3.3245599269866943


In [455]:
from watermarking.experiments.watermark import compute_ppl_single

# 计算带time水印的ppl
loss, ppl = compute_ppl_single(prefix_and_output_text=t_prefix_and_output_text,
                               oracle_model_name=args.oracle_model_name,
                               output_text=t_output_text,
                               oracle_model=oracle_model, oracle_tokenizer=oracle_tokenizer)
print(loss, ppl)

1.1480250358581543 3.1519618034362793


In [424]:
# 计算不带水印的ppl
loss, ppl = compute_ppl_single(prefix_and_output_text=no_wm_prefix_and_output_text,
                               oracle_model_name=args.oracle_model_name,
                               output_text=no_wm_output_text,
                               oracle_model=oracle_model, oracle_tokenizer=oracle_tokenizer)
print(loss, ppl)

0.9888984560966492 2.6882715225219727
